# Fine Tuning DistilBERT for Movie Genre Classification

### Introduction

**This script is an adaptation of a tutorial on Fine Tuning DistilBERT on the Toxic data. Here is the link to the oiriginal [tutorial](https://colab.research.google.com/github/DhavalTaunk08/Transformers_scripts/blob/master/Transformers_multilabel_distilbert.ipynb) available on [Hugging Face](https://huggingface.co/).**

In this tutorial we will be fine tuning a transformer model for the **Multilabel text classification** problem.
We will use the existing base, with some changes, to predict the genre of a movie based on the synopsis.

#### Flow of the notebook


1. [Importing Python Libraries and preparing the environment](#section01)
2. [Importing and Pre-Processing the domain data](#section02)
3. [Preparing the Dataset and Dataloader](#section03)
4. [Creating the Neural Network for Fine Tuning](#section04)
5. [Fine Tuning the Model](#section05)
6. [Validating the Model Performance](#section06)
7. [Saving the model and artifacts for Inference in Future](#section07)
8. [Set up a quick example of how the model classifies a new synopsis](#Section08)

#### Technical Details


 - Data:
	 - We are using a dataset fetched from [The Movie DataBase](https://www.themoviedb.org/), originally containing movie Title, Synopsis, and Genre.

Each movie can be assigned to multiple genres.

 - Language Model Used:
	 - DistilBERT is a smaller transformer model as compared to BERT or Roberta. It is created by process of distillation applied to Bert.  
	 - [Blog-Post](https://medium.com/huggingface/distilbert-8cf3380435b5)
	 - [Research Paper](https://arxiv.org/pdf/1910.01108)
     - [Documentation for python](https://huggingface.co/transformers/model_doc/distilbert.html)


 - Script Objective:
	 - The objective of this script is to fine tune DistilBERT to be able to label a movie synopsis into the following genres:

		 - `Thriller`
		 - `Horror`
		 - `Drama`
		 - `Comedy`
		 - `Romance`
		 - `Science Fiction`
		 - `Adventure`
		 - `Action`

---
***NOTE***
- *It is to be noted that the overall mechanisms for a multiclass and multilabel problems are similar, except for few differences namely:*
	- *Loss function is designed to evaluate all the probability of categories individually rather than as compared to other categories. Hence the use of `BCE` rather than `Cross Entropy` when defining loss.*
	- *Sigmoid of the outputs calculated to rather than Softmax. Again for the reasons defined in the previous point*
	- *The [loss metrics](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.hamming_loss.html) and **Hamming Score**  are used for direct comparison of expected vs predicted*
---

<a id='section01'></a>
### Importing Python Libraries and preparing the environment

At this step we will be importing the libraries and modules needed to run our script. Libraries are:
* warnings
* Numpy
* Pandas
* tqdm
* scikit-learn metrics
* Pytorch
* Pytorch Utils for Dataset and Dataloader
* Transformers
* DistilBERT Model and Tokenizer
* logging

Followed by that we will preapre the device for CUDA execeution. This configuration is needed if you want to leverage on onboard GPU.

In [1]:
! pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.9/7.9 MB 31.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 37.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 75.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 74.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 40.7 MB/s eta 0:00:00


In [2]:
# Importing stock ml libraries
import warnings
warnings.simplefilter('ignore')
import numpy as np
import pandas as pd
from tqdm import tqdm
from sklearn import metrics
import transformers
import torch
from torch.utils.data import Dataset, DataLoader, RandomSampler, SequentialSampler
from transformers import DistilBertTokenizer, DistilBertModel
import logging
logging.basicConfig(level=logging.ERROR)

In [3]:
# # Setting up the device for GPU usage

from torch import cuda
device = 'cuda' if cuda.is_available() else 'cpu'

In [6]:
def hamming_score(y_true, y_pred, normalize=True, sample_weight=None):
    acc_list = []
    for i in range(y_true.shape[0]):
        set_true = set( np.where(y_true[i])[0] )
        set_pred = set( np.where(y_pred[i])[0] )
        tmp_a = None
        if len(set_true) == 0 and len(set_pred) == 0:
            tmp_a = 1
        else:
            tmp_a = len(set_true.intersection(set_pred))/\
                    float( len(set_true.union(set_pred)) )
        acc_list.append(tmp_a)
    return np.mean(acc_list)

<a id='section02'></a>
### Importing and Pre-Processing the domain data

In [7]:
data = pd.read_csv('movies_df.csv')

In [8]:
data.head()

,title,synopsis,Adventure,Action,Romance,Thriller,Drama,Horror,Science Fiction,Comedy
0,Expend4bles,Armed with every weapon they can get their han...,1,1,0,1,0,0,0,0
1,Mission: Impossible - Dead Reckoning Part One,Ethan Hunt and his IMF team embark on their mo...,0,1,0,1,0,0,0,0
2,The Equalizer 3,Robert McCall finds himself at home in Souther...,0,1,0,1,0,0,0,0
3,Mortal Kombat Legends: Cage Match,"In 1980s Hollywood, action star Johnny Cage is...",0,1,0,0,0,0,0,0
4,Desperation Road,"After 11 years in a Mississippi state prison, ...",0,1,0,1,1,0,0,0


In [9]:
data.shape

(30329, 10)

In [10]:
# data = data.drop('title',axis=1)

In [11]:
# data.drop(['id'], inplace=True, axis=1)

In [12]:
new_df = pd.DataFrame()
new_df['synopsis'] = data['synopsis']
new_df['labels'] = data.iloc[:, 2:].values.tolist()

In [13]:
new_df.head()

,synopsis,labels
0,Armed with every weapon they can get their han...,"[1, 1, 0, 1, 0, 0, 0, 0]"
1,Ethan Hunt and his IMF team embark on their mo...,"[0, 1, 0, 1, 0, 0, 0, 0]"
2,Robert McCall finds himself at home in Souther...,"[0, 1, 0, 1, 0, 0, 0, 0]"
3,"In 1980s Hollywood, action star Johnny Cage is...","[0, 1, 0, 0, 0, 0, 0, 0]"
4,"After 11 years in a Mississippi state prison, ...","[0, 1, 0, 1, 1, 0, 0, 0]"


<a id='section03'></a>
### Preparing the Dataset and Dataloader

We will start with defining few key variables that will be used later during the training/fine tuning stage.
Followed by creation of MultiLabelDataset class - This defines how the text is pre-processed before sending it to the neural network. We will also define the Dataloader that will feed  the data in batches to the neural network for suitable training and processing.
Dataset and Dataloader are constructs of the PyTorch library for defining and controlling the data pre-processing and its passage to neural network. For further reading into Dataset and Dataloader read the [docs at PyTorch](https://pytorch.org/docs/stable/data.html)

#### *MultiLabelDataset* Dataset Class
- This class is defined to accept the `tokenizer`, `dataframe` and `max_length` as input and generate tokenized output and tags that is used by the BERT model for training.
- We are using the DistilBERT tokenizer to tokenize the data in the `text` column of the dataframe.
- The tokenizer uses the `encode_plus` method to perform tokenization and generate the necessary outputs, namely: `ids`, `attention_mask`, `token_type_ids`

- To read further into the tokenizer, [refer to this document](https://huggingface.co/transformers/model_doc/distilbert.html#distilberttokenizer)
- `targets` is the list of categories labled as `0` or `1` in the dataframe.
- The *MultiLabelDataset* class is used to create 2 datasets, for training and for validation.
- *Training Dataset* is used to fine tune the model: **80% of the original data**
- *Validation Dataset* is used to evaluate the performance of the model. The model has not seen this data during training.

#### Dataloader
- Dataloader is used to for creating training and validation dataloader that load data to the neural network in a defined manner. This is needed because all the data from the dataset cannot be loaded to the memory at once, hence the amount of dataloaded to the memory and then passed to the neural network needs to be controlled.
- This control is achieved using the parameters such as `batch_size` and `max_len`.
- Training and Validation dataloaders are used in the training and validation part of the flow respectively

In [14]:
# Sections of config

# Defining some key variables that will be used later on in the training
MAX_LEN = 512
TRAIN_BATCH_SIZE = 20
VALID_BATCH_SIZE = 20
EPOCHS = 2
LEARNING_RATE = 1e-05
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased', truncation=True, do_lower_case=True)

In [15]:
class MultiLabelDataset(Dataset):

    def __init__(self, dataframe, tokenizer, max_len):
        self.tokenizer = tokenizer
        self.data = dataframe
        self.synopsis = dataframe.synopsis
        self.targets = self.data.labels
        self.max_len = max_len

    def __len__(self):
        return len(self.synopsis)

    def __getitem__(self, index):
        synopsis = str(self.synopsis[index])
        synopsis = " ".join(synopsis.split())

        inputs = self.tokenizer.encode_plus(
            synopsis,
            None,
            add_special_tokens=True,
            max_length=self.max_len,
            pad_to_max_length=True,
            truncation=True
        )
        ids = inputs['input_ids']
        mask = inputs['attention_mask']


        return {
            'ids': torch.tensor(ids, dtype=torch.long),
            'mask': torch.tensor(mask, dtype=torch.long),
            'targets': torch.tensor(self.targets[index], dtype=torch.float)
        }

In [16]:
# Creating the dataset and dataloader for the neural network

train_size = 0.8
train_data=new_df.sample(frac=train_size,random_state=200)
test_data=new_df.drop(train_data.index).reset_index(drop=True)
train_data = train_data.reset_index(drop=True)


print("FULL Dataset: {}".format(new_df.shape))
print("TRAIN Dataset: {}".format(train_data.shape))
print("TEST Dataset: {}".format(test_data.shape))

training_set = MultiLabelDataset(train_data, tokenizer, MAX_LEN)
testing_set = MultiLabelDataset(test_data, tokenizer, MAX_LEN)

FULL Dataset: (30329, 2)
TRAIN Dataset: (24263, 2)
TEST Dataset: (6066, 2)


In [17]:
train_params = {'batch_size': TRAIN_BATCH_SIZE,
                'shuffle': True,
                'num_workers': 0
                }

test_params = {'batch_size': VALID_BATCH_SIZE,
                'shuffle': True,
                'num_workers': 0
                }

training_loader = DataLoader(training_set, **train_params)
testing_loader = DataLoader(testing_set, **test_params)

<a id='section04'></a>
### Creating the Neural Network for Fine Tuning

#### Neural Network
 - We will be creating a neural network with the `DistilBERTClass`.
 - This network will have the `DistilBERT` model.  Follwed by a `Droput` and `Linear Layer`. They are added for the purpose of **Regulariaztion** and **Classification** respectively.
 - In the forward loop, there are 2 output from the `DistilBERTClass` layer.
 - The second output `output_1` or called the `pooled output` is passed to the `Drop Out layer` and the subsequent output is given to the `Linear layer`.
 - Keep note the number of dimensions for `Linear Layer` is **6** because that is the total number of categories in which we are looking to classify our model.
 - The data will be fed to the `DistilBERTClass` as defined in the dataset.
 - Final layer outputs is what will be used to calcuate the loss and to determine the accuracy of models prediction.
 - We will initiate an instance of the network called `model`. This instance will be used for training and then to save the final trained model for future inference.

#### Loss Function and Optimizer
 - The Loss is defined in the next cell as `loss_fn`.
 - As defined above, the loss function used will be a combination of Binary Cross Entropy which is implemented as [BCELogits Loss](https://pytorch.org/docs/stable/nn.html#bcewithlogitsloss) in PyTorch
 - `Optimizer` is defined in the next cell.
 - `Optimizer` is used to update the weights of the neural network to improve its performance.

In [18]:
# Creating the customized model, by adding a drop out and a dense layer on top of distil bert to get the final output for the model.

class DistilBERTClass(torch.nn.Module):
    def __init__(self):
        super(DistilBERTClass, self).__init__()
        self.l1 = DistilBertModel.from_pretrained("distilbert-base-uncased")
        self.pre_classifier = torch.nn.Linear(768, 768)
        self.dropout = torch.nn.Dropout(0.1)
        self.classifier = torch.nn.Linear(768, 8)

    def forward(self, input_ids, attention_mask):
        output_1 = self.l1(input_ids=input_ids, attention_mask=attention_mask)
        hidden_state = output_1[0]
        pooler = hidden_state[:, 0]
        pooler = self.pre_classifier(pooler)
        pooler = torch.nn.Tanh()(pooler)
        pooler = self.dropout(pooler)
        output = self.classifier(pooler)
        return output

model = DistilBERTClass()
model.to(device)

DistilBERTClass(
  (l1): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0-5): 6 x TransformerBlock(
          (attention): MultiHeadSelfAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0.1, inplace=False)
            (lin1): Linear(in

In [19]:
def loss_fn(outputs, targets):
    return torch.nn.BCEWithLogitsLoss()(outputs, targets)

In [20]:
optimizer = torch.optim.Adam(params =  model.parameters(), lr=LEARNING_RATE)

<a id='section05'></a>
### Fine Tuning the Model

After all the effort of loading and preparing the data and datasets, creating the model and defining its loss and optimizer. This is probably the easier steps in the process.

Here we define a training function that trains the model on the training dataset created above, specified number of times (EPOCH), An epoch defines how many times the complete data will be passed through the network.

Following events happen in this function to fine tune the neural network:
- The dataloader passes data to the model based on the batch size.
- Subsequent output from the model and the actual category are compared to calculate the loss.
- Loss value is used to optimize the weights of the neurons in the network.
- After every 5000 steps the loss value is printed in the console.

As you can see just in 1 epoch by the final step the model was working with a miniscule loss of 0.05 i.e. the network output is extremely close to the actual output.

In [21]:
def train(epoch):
    model.train()
    for _,data in tqdm(enumerate(training_loader, 0)):
        ids = data['ids'].to(device, dtype = torch.long)
        mask = data['mask'].to(device, dtype = torch.long)
        targets = data['targets'].to(device, dtype = torch.float)

        outputs = model(ids, mask)

        optimizer.zero_grad()
        loss = loss_fn(outputs, targets)
        if _%5000==0:
            print(f'Epoch: {epoch}, Loss:  {loss.item()}')

        loss.backward()
        optimizer.step()

In [22]:
for epoch in range(EPOCHS):
    train(epoch)

0it [00:00, ?it/s]

Epoch: 0, Loss:  0.6942421793937683


1214it [17:25,  1.16it/s]
0it [00:00, ?it/s]

Epoch: 1, Loss:  0.28985798358917236


1214it [17:27,  1.16it/s]
0it [00:00, ?it/s]

Epoch: 2, Loss:  0.2943437993526459


1214it [17:26,  1.16it/s]




```
# This is formatted as code
```

<a id='section06'></a>
### Validating the Model

During the validation stage we pass the unseen data(Testing Dataset) to the model. This step determines how good the model performs on the unseen data.

This unseen data is the 20% of the original data, which was separated during the Dataset creation stage.
During the validation stage the weights of the model are not updated. Only the final output is compared to the actual value. This comparison is then used to calcuate the accuracy of the model.

As defined above to get a measure of our models performance we are using the following metrics.
- Hamming Score
- Hamming Loss


In [23]:
def validation(testing_loader):
    model.eval()
    fin_targets=[]
    fin_outputs=[]
    with torch.no_grad():
        for _, data in tqdm(enumerate(testing_loader, 0)):
            ids = data['ids'].to(device, dtype = torch.long)
            mask = data['mask'].to(device, dtype = torch.long)
            targets = data['targets'].to(device, dtype = torch.float)
            outputs = model(ids, mask)
            fin_targets.extend(targets.cpu().detach().numpy().tolist())
            fin_outputs.extend(torch.sigmoid(outputs).cpu().detach().numpy().tolist())
    return fin_outputs, fin_targets

In [24]:
outputs, targets = validation(testing_loader)

final_outputs = np.array(outputs) >=0.5

304it [01:40,  3.03it/s]


In [25]:
from sklearn.metrics import precision_score, recall_score, f1_score

In [26]:
print(f'Precision score: {precision_score(targets, final_outputs, average="weighted"):.2f}')
print(f'Recall score:    {recall_score(targets, final_outputs, average="weighted"):.2f}')
print(f'F1 score:        {f1_score(targets, final_outputs, average="weighted"):.2f}')

Precision score: 0.73
Recall score:    0.63
F1 score:        0.68


In [27]:
val_hamming_loss = metrics.hamming_loss(targets, final_outputs)
val_hamming_score = hamming_score(np.array(targets), np.array(final_outputs))

print(f"Hamming Score = {val_hamming_score}")
print(f"Hamming Loss = {val_hamming_loss}")

Hamming Score = 0.5713759753819101
Hamming Loss = 0.1511086383119024


<a id='section07'></a>
### Saving the Trained Model for inference

This is the final step in the process of fine tuning the model.

The model and its vocabulary are saved locally. These files are then used in the future to make inference on new inputs of news headlines.

In [28]:
# Saving the files for inference

output_model_file = 'pytorch_distilbert_movies.bin'
output_vocab_file = 'vocab_distilbert_movies.bin'

torch.save(model, output_model_file)
tokenizer.save_vocabulary(output_vocab_file)

print('Saved')

Saved


In [29]:
# check predictions and establish why model is not performing very well (more epochs, more data)?

<a id='section08'></a>
### Testing the model with previously unseen synopses
Here we create a predict_genre function that will allow us to check the performance of the model with previously unseen synopses.

In [30]:
def predict_genre(model, tokenizer, synopsis, device, threshold=0.5):
    # Tokenize the synopsis
    inputs = tokenizer.encode_plus(
        synopsis,
        add_special_tokens=True,
        max_length=256,  # same max_length as used earlier
        padding='max_length',
        truncation=True,
        return_tensors="pt"  # return tensors format
    )

    input_ids = inputs["input_ids"].to(device)
    attention_mask = inputs["attention_mask"].to(device)

    # Get model's output
    with torch.no_grad():
        logits = model(input_ids, attention_mask)

    # Convert logits to probabilities
    probs = torch.sigmoid(logits).cpu().numpy()
    predictions = (probs > threshold).astype(int)  # threshold to get class predictions

    return predictions[0], probs[0]

In [31]:
# Test the function
model.eval()  # set model to evaluation mode
#new_synopsis = "Muldron is a planet in galaxy 577, torn apart by long wars between the Pistis and the Bolzi. New problems arise when an alien invasion occur"
new_synopsis = "Charlie is a hitman working for the Russian Mafia. Will he find his way home?"
predicted_labels, predicted_probs = predict_genre(model, tokenizer, new_synopsis, device)

In [32]:
genres = ['Adventure','Action',	'Romance','Thriller',	'Drama','Horror',	'Science Fiction','Comedy']

In [33]:
predicted_probs = list(predicted_probs)

In [34]:
output = [round(i) for i in predicted_probs]

In [35]:
output

[0, 1, 0, 1, 0, 0, 0, 0]

In [36]:
print("Predicted Labels:", predicted_labels)
print("Predicted Probabilities:", predicted_probs)

Predicted Labels: [0 1 0 1 0 0 0 0]
Predicted Probabilities: [0.10768034, 0.7999737, 0.05456449, 0.7247405, 0.22600773, 0.010647607, 0.0048921607, 0.21178399]


In [37]:
print("The predicted label for this synopsis is: " + str([j[1] for j in zip(output,genres) if j[0]==1]))

The predicted label for this synopsis is: ['Action', 'Thriller']
